In [1]:
import ollama
import pandas as pd
from typing import Dict
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain_core.runnables import RunnableSequence
from pydantic import BaseModel, Field, field_validator
from IPython.display import Markdown, display, update_display
from langchain_core.output_parsers import PydanticOutputParser

#### **Let's make a CLASS to curate NOVELTY:**

In [38]:
class Curate_Features:
    def __init__(self, model: str = "qwen2.5:14b-instruct", temperature: float = 0.0):
        self.llm = ChatOpenAI(
            base_url="http://localhost:11434/v1/",
            api_key="ollama",
            max_completion_tokens=1100,
            max_retries=3,
            model=model,
            temperature=temperature,
        )

    def curate_novelty(self, user_prompt: str) -> Dict[str, str]:
        system_prompt = f'''
        You are an AI assistant specializing in curating the NOVELTY of patents based on the provided TITLE, ABSTRACT and FirstClaim. 
        The NOVELTY should describe what makes the patent novel—highlighting the unique features and innovations of the patent—without exceeding 1000 characters.
        
        Definations:
        - SCOPE (What the invention is): is defined as the subject of the invention which relates to the FirstClaim. The scope may not be fully described in the FirstClaim and the curator may need to refer to other parts of the patent specification in order to provide a complete description.
        - USE: is defined as the specific application(s) of the invention.
        - NOVELTY: is defined as the new feature(s) or step(s) introduced by the inventor which is not conventional and an improvement on existing technology in a clear and concise form. Alternatively, the novelty can be described as the unique inventive feature of the invention.
        - ADVANTAGE: is defined as the improvement over prior art or existing inventions and technology as a result novelty of the invention.
        
        Rules for curating NOVELTY:
        - Novelty will not exceed a length of 1000 characters
        - Novelty will not state advantages.
        - Novelty will not state prior art.
        - All chemical compounds in the Novelty will be at least partially named.
        - When two or more equal main claims are identified, the information from both will be covered in the Novelty section
        - Novelty will not run on from the title.
        - Novelty will include figure references for parts labels. These are usally written in parentheses example, (3) or (4).
        - Novelty excludes the following:
          1. Embedded formulae
          2. Markush definitions
          3. Sequence definitions:
                - For sequences of <= 30 nucleobases or <= 10 amino acids:
                The actual sequence must be excluded from Novelty.
                Reference only as "a sequence of X amino acids" or "a sequence of X nucleobases".
                - For sequences >30 nucleobases or >10 amino acids:
                Use the phrase "having a defined sequence of [length], given in the specification".
                
                Examples:
                - Short Sequence Example:
                
                  Claim: "...the affinity agent comprising 10 amino acids of sequence LEKGAMTSPQ (SEQ ID NO:1)..."
                  
                  Novelty: "...the affinity agent comprising a sequence of 10 amino acids (SEQ ID NO: 1)..."
                
                - Long Sequence Example:
                
                  Claim: "...comprising a protein sequence of 785 amino acids (SEQ ID NO:1)..."
                  
                  Novelty: "...comprising a protein having a defined sequence of 785 amino acids (SEQ ID NO: 1), given in the specification..."
        
        Guidelines:
        - Priority: The FirstClaim should be given the highest importance when identifying the NOVELTY. Focus on the core claims and their unique aspects.
        - ABSTRACT Integration: If any novel features are mentioned in the ABSTRACT but not fully captured in the FirstClaim, include them, but ensure the focus remains on the NOVELTY.
        - Accuracy: Pay special attention to part labels (e.g., "(3)") in both the ABSTRACT and FirstClaim. These must be transcribed highly accurately in the curated NOVELTY. Include some of the important part labels. Prioritize conciseness and accuracy of part labels, but do not repeat them.
        - Intelligent Summarization: Curate the NOVELTY with enough detail to fully explain what makes the patent unique without repeating information. You need to be concise while ensuring that all essential features of the invention are included.
        - Character Limit: The NOVELTY must stay under 1000 characters. Ensure it includes all necessary information, but concise writing is crucial. Avoid excessive elaboration or repetition.
        
        Structure:
        - Introduction: Briefly introduce the invention, summarizing its core concept and main innovation.
        - Novel Features: Identify and describe the specific aspects of the patent that make it novel.
        
        Focus on the following:
        - Design, method, or system innovations.
        - Key components that differentiate it from prior art.
        - Unique use of technology or novel combinations of technologies.
        - Detailed features found in the FirstClaim that distinguish this invention.
        - Mention any novelty highlighted in the ABSTRACT if relevant.
        - ABSTRACT Integration: If any part of the ABSTRACT provides additional NOVELTY, include it without redundancy, and ensure the overall NOVELTY is cohesive.
        
        EXAMPLE 1:
        
        TITLE:
        An assembly for building.
        
        ABSTRACT:
        The application claims a building component, comprising: a supporting piece, a pair of first sealing piece, an elastic deformable gasket and a second sealing piece. The supporting piece is provided with a bottom part, a channel located at the centre of the bottom part, and a pair of tracks located at two sides of the channel. Each first sealing element comprises a first bottom and two skid-proof parts respectively outwards from the first bottom. the elastic deformable gasket has a space with a U-shaped section or a V-shaped section, and two flanges respectively outwards horizontally from the U-shaped or V-shaped space edge. The second sealing element comprises a second bottom and a rib set in the centre of the second bottom. In addition, the pair of first sealing piece can be connected with the supporting piece meshing the elastic deformable pad can be clamped in the channel of the supporting piece; the second sealing piece is pressed into the space of U-shaped or V-shaped section by the rib, and the elastic deformable gasket meshing and when the pair of first sealing piece and the supporting piece meshing, it can create two pocket areas capable of being used for mounting the building wall plate. 
        
        FirstClaim:
        An assembly for construction, wherein it comprises: a supporting member, comprising: a bottom plate, which has a front side and a back side; a channel, located at the centre of the front side of the bottom plate; and a pair of rails, located respectively the channel; wherein the channel and the pair of tracks respectively extended along the length direction of the bottom plate and have the same length as the bottom plate; a pair of first sealing piece, each first sealing piece comprises: a first bottom, which has a front side and a back side; two anti-sliding parts, respectively extend along the length direction from the first bottom back and have the same length with the first bottom length; an elastic deformable gasket, having a space with a U-shaped section or a V-shaped section; and two flanges, respectively extends outwards from the edge level of the space with U-shaped or V-shaped cross section; and a second sealing element, comprising: a second bottom, which has a front side and a back side; and a rib, located at the centre of the rear side of the second bottom and extending along the length direction thereof and having the same length as the length of the second bottom; wherein the pair of first sealing elements is meshing into the rail of the supporting element by pressing the two anti-sliding portions of each first sealing element into the rail of the supporting element; the elastic deformable gasket can be clamped in the channel of the (snap-in) supporting piece; the second sealing piece is pressed into the space of the U-shaped or V-shaped section of the elastic deformable gasket by pressing the rib, and the elastic deformable gasket meshing and when the pair of first sealing piece and the supporting piece meshing, it can create two pocket respectively of containing building wall plate in it. 
        
        NOVELTY:
        The assembly (100) has a bottom plate provided with a front side and a back side. A channel is located at a center of the front side of the bottom plate. A pair of rails are located on the channel. The channel and the pair of tracks are respectively extended along length direction of the base plate and have the same length as the bottom board. The pair of first sealing elements (120) is meshed into a rail of a supporting element by pressing two anti-sliding portions of each first sealing element into the rail of the supporting element. An elastic deformable gasket is clamped in the channel of the supporting piece. A second sealing element is pressed into a space of a U-shaped or V-shaped section of the elastic deformable gasket by pressing the rib. 
        
        END OF EXAMPLE 1
        
        EXAMPLE 2:
        
        TITLE:
        An agricultural LED lamp convenient to use 
        
        ABSTRACT:
        The invention claims an agricultural LED lamp convenient for use, comprising a base; the inner part of the sliding cavity is slidingly connected with a convex rod; and the bottom of the convex rod is in threaded connection with the inner ring of the gear plate; the top end of the lamp rod is provided with a limiting groove; the inner part of the limiting groove is fixedly installed with an inner plate; the top end of the inner plate is fixedly installed with a spring; the inner ring of the spring is fixedly sleeved with a limiting rod; two sides of the bottom end of the top plate are fixedly mounted with an LED lamp; the bottom ends of the two LED lamps are fixedly connected with a collecting plate; the inner part of the two collecting plates are fixedly provided with a collecting groove; the invention claims an agricultural LED lamp convenient for use, a rotating gear disc; driving the convex rod to rotate and penetrate into the ground, reinforcing the stability of the base, and convenient to fix; the top part through the matching of the limiting rod and the limiting groove can fix the top plate and the LED lamp; the mounting is convenient, and the limiting groove is provided with a spring; it has buffer performance, preventing the damage of the parts in the LED lamp. 
        
        FirstClaim:
        An agricultural LED lamp convenient for use, comprising a base (1), wherein the inner part of the base (1) is fixedly installed with a gear plate (12); the top end of the base (1) is fixedly installed with a lamp rod (15); the inner part of the lamp rod (15) is provided with a sliding cavity (14); the inner part of the sliding cavity (14) is slidingly connected with a convex rod (13), and the bottom of the convex rod (13) is connected with the inner ring of the gear plate (12); the top end of the lamp rod (15) is provided with a limiting groove (11); the inner part of the limiting groove (11) is fixedly installed with an inner plate (7); the top end of the inner plate (7) is fixedly mounted with a spring (8); the inner ring of the spring (8) is fixedly sleeved with a limiting rod (10); and the top end of the limiting rod (10) is fixedly connected with the middle part of the bottom end of the top plate (6); the two sides of the bottom end of the top plate (6) are fixedly mounted with an LED lamp (5); the bottom end of the two LED lamps (5) are fixedly connected with a collecting plate (16); the inner part of the two collecting plates (16) are fixedly provided with a collecting groove. 
        
        NOVELTY:
        The lamp has a base (1) whose inner part is fixedly installed with a gear plate (12). A top end of the base is installed with a lamp rod. An inner part of the lamp rod is formed with a sliding cavity. An inner part of the sliding cavity is slidingly connected with a convex rod (13). A bottom part of the convex rod is connected with an inner ring of the gear plate. A top end of the lamp rod is formed with a limiting groove (11). A bottom end of two light emitted diode lamps (5) are fixedly connected with a collecting plate. An inner part of the collecting plate is formed with a collecting groove.
        
        END OF EXAMPLE 2
        
        EXAMPLE 3:
        
        TITLE:
        False news countermeasure detection system and method for linking external knowledge base. 
        
        ABSTRACT:
        The invention claims a false news countermeasure detection system and method for linking external knowledge base, wherein the system comprises a multi-mode characteristic extraction unit, an external knowledge base comparison unit and a classification identification model, wherein the classification identification model comprises an event classifier and a false news detection unit; the multimodal feature extraction unit comprises a text multimodal feature extraction layer, an image multimodal feature extraction layer and a full connection layer; the external knowledge base comparing unit comprises an entity representation and an entity comparison, the entity representation comprises a text representation and an image representation; the event classifier comprises a full connection layer and a corresponding activation function connected with the multi-mode characteristic extracting unit; The false news detection unit comprises a full connection layer connected with the multimodal feature extraction unit. The invention improves the accuracy of the mixed news mutual identification of the Chinese and English texts by fusing the multi-mode characteristic link external knowledge base. 
        
        FirstClaim:
        A false news countermeasure detection system linked with external knowledge base, wherein the system comprises a multimodal feature extraction unit, an external knowledge base comparison unit and a classification identification model, the classification identification model comprises an event classifier and a false news detection unit; the multi-modal feature extracting unit comprises a text multi-modal feature extracting layer, an image multi-modal feature extracting layer and a full connecting layer; the external knowledge base comparing unit comprises an entity representation and an entity comparison, the entity representation comprises a text representation and an image representation; the event classifier comprises a full connection layer and a corresponding activation function, which is connected with the multimodal feature extraction unit for classifying different events and evaluating the similarity of different events; the false news detecting unit comprises a full connecting layer connected with the multi-mode characteristic extracting unit for searching the optimal parameter to finish the true and false identification of the Chinese-English mixed news; The loss function of the classification identification model is as follows: Lfinal (theta f, theta d, theta e) = Ld (theta f, theta d) - lambda Le (theta f, theta e), wherein Le (theta f, theta e) represents the classification loss of the event classifier, Ld (theta f, theta d) represents the detection loss of the false news detector, lambda represents the weight parameter between the false news detection loss function and the event classification loss function. 
        
        NOVELTY:
        The false news countermeasure detection system comprises a classification identification model that has an event classifier and a false news detection unit. A multi-modal feature extracting unit provides a text multi-modal feature extracting layer, an image multi-modal feature extracting layer, and a full connecting layer. An external knowledge base comparing unit provides an entity representation and an entity comparison. The entity representation has a text representation and an image representation. The event classifier provides a full connection layer and a corresponding activation function, which is connected with the multimodal feature extraction unit for classifying different events and evaluating the similarity of different events. The false news detection unit provides a full connecting layer that is connected with a multi-mode characteristic extracting unit for searching the optimal parameter to finish the true and false identification of the Chinese-English mixed news.
        
        END OF EXAMPLE 3
        
        EXAMPLE 4:
        
        TITLE:
        A hollow ring connected multi-blade stirrer. 
        
        ABSTRACT:
        The invention claims a hollow ring connected multi-blade stirrer, belonging to the technical field of stirrer The hollow ring connecting multi-blade stirrer comprises central shaft; the number of the hollow ring is multiple, each hollow ring is set outside the central shaft along the length direction of the central shaft; the number of the stirring blade is more than one, the stirring blade is uniformly set on the circumferential direction of the hollow ring; the connecting blade is used for passing through the through hole set on the central shaft and connected with two stirring blades symmetrically distributed in axial direction. The hollow ring of the invention is connected with multiple blades stirrer the invention claims a hollow ring connected with multiple stirrer of large reinforced fluid uniform mixing or multi-phase fluid uniformly dispersed, radial flow and axial flow interaction strengthens the fluid exchange in the kettle, so that the shearing force and turbulent energy are uniformly distributed, improving the flowing state in the kettle, improving fluid mixing efficiency, reducing the consumption of the stirrer, at the same time, the structure of the hollow ring connected with multiple blades stirrer simple, easy to process, and reducing the manufacturing cost. 
        
        FirstClaim:
        A hollow ring connected with multiple stirrer, wherein it comprises: a central shaft (1); hollow ring (2), the number is multiple, each hollow ring (2) along the length direction of the central shaft (1) is set on the outer part of the central shaft (1); stirring blade (3), the number is multiple, uniformly set on the circumferential direction of the hollow ring (2); and a connecting blade (4) for passing through the through hole set on the central shaft (1) and connecting two stirring blades (3) symmetrically distributed in the axial direction. 
        
        NOVELTY:
        The ring has a hollow ring (2) fixed on an outer side of a central shaft (1) along length direction. Two stirring blades (3) are uniformly fixed on circumferential direction of the hollow ring and symmetrically distributed in an axial direction. A connecting blade (4) passes through a through hole that is fixed on the central shaft. The hollow ring is provided with a first hollow ring and a second hollow ring that are connected with each other. The central shaft is connected with the connecting blade through the through hole. 
        
        END OF EXAMPLE 4

        Input Text:
        Curate the NOVELTY from the provided information while ensuring it remains within 1,000 characters. Include some of the important part labels. Prioritize conciseness and accuracy of part labels, but do not repeat them.
        {{user_prompt}}
        
        OUTPUT REQUIREMENTS:
        
        The response should only contain one JSON object with just one key:
        - "novelty": A single string containing the complete curated text. Ensure that the length of this text should not exceed 1000 characters i.e., len(text) <= 1000.
        
        The output should not contain nested structures or dictionaries.
        
        The output must be in valid JSON format.
        '''
        
        class enforce_output_format(BaseModel):
            novelty: str = Field(description="Complete curated novelty text.")

            @field_validator("novelty")
            @classmethod
            def enforce_character_limit(cls, value: str) -> str:
                character_count = len(value)
                if character_count > 1000:
                    raise ValueError(f"Output exceeded the character limit. (Got {character_count} characters).")
                return value      

            def to_dict(self) -> Dict[str, str]:
                return {"novelty": self.novelty}

        output_parser = PydanticOutputParser(pydantic_object=enforce_output_format)

        prompt_template = PromptTemplate(input_variables=["user_prompt"], template=system_prompt)
        summary_chain = prompt_template | self.llm | output_parser
        result = summary_chain.invoke({"user_prompt": user_prompt})
        return result.to_dict()

In [39]:
curator = Curate_Features(model = "deepseek-r1:14b") # model = qwen2.5:14b-instruct or phi4 or deepseek-r1:14b or llama3.2:3b-instruct-fp16

# This is a tough example where SUMMIT will always truncate the Novelty to keep it within 1000 characters
user_prompt = '''
TITLE:
An automatic laminating machine for auxiliary material.

ABSTRACTS:
The invention belongs to the field of auxiliary material laminating machine, specifically to an auxiliary material automatic laminating machine, comprising a working table; the middle part of the top side of the working table is fixedly connected with an electric slide rail; the electric slide rail is provided with a locating fixture; the top side of the working table is fixedly connected with a bracket; the middle part of the bracket is rotatably connected with a screw rod; the end part of the screw rod is fixedly connected with a servo motor; said servo motor is fixedly connected with the bracket; the screw rod is connected with a top plate through a ball screw nut pair; the output end of the cylinder is fixedly connected with a bonding plate: multiple vacuum suckers are arranged at the bottom of the bonding plate; the bonding plate is located at the top part of the locating jig, the bubble of the protective film and the product is driven out from the centre to two sides by the movement of the bubble removing roller, so as to reduce the bubble in the middle part of the protective film, so as to reduce the condition that the bubble in the middle part of the protective film is difficult to discharge after the pressure maintaining bonding device main body is pressed and bonded, Therefore, the bubble after the protection film is adhered is reduced. 

FirstClaim:
An auxiliary material automatic laminating machine, comprising a working table (11); the middle part of the top side of the working table (11) is fixedly connected with an electric slide rail (13); the electric slide rail (13) is provided with a locating jig (14); the top side of the working table (11) is fixedly connected with a bracket (12); the middle part of the bracket (12) is rotatably connected with a screw rod; the end part of the screw rod is fixedly connected with a servo motor (15); the servo motor (15) is fixedly connected with the bracket (12); the screw rod is connected with a top plate (16) through a ball screw nut pair; the top plate (16) is fixedly connected with a cylinder (161); the output end of the cylinder (161) is fixedly connected with a bonding plate (17); wherein: the bottom of the bonding plate (17) is provided with a plurality of vacuum suction discs (162); the bonding plate (17) is located on the top of the locating jig (14); the two sides of the bonding plate (17) are slidingly connected with a pair of sliding seat (19); the bottom of the sliding seat (19) is fixedly connected with a lifting spring; the bottom end of the lifting spring is fixedly connected with a fixing seat (191); the bottom of the fixing seat (191) is rotatably connected with a de-foaming roller (193); the side of the bonding plate (17) is fixedly connected with an electric push rod (192); the output end of the electric push rod (192) is fixedly connected with the sliding seat (19).
'''

result = curator.curate_novelty(user_prompt=user_prompt)

In [36]:
# Qwen2.5 14b Instruct
display(Markdown(result['novelty']))

An automatic laminating machine for auxiliary material, comprising a working table (11); the middle part of the top side of the working table is fixedly connected with an electric slide rail (13) and provided with a locating jig (14). A bracket (12) is fixed on the top side of the working table. The screw rod in the bracket is rotatably connected to a servo motor (15), which drives a ball screw nut pair connecting to a top plate (16). The cylinder (161) on the top plate connects to a bonding plate (17) with vacuum suction discs (162). Sliding seats (19) are connected to both sides of the bonding plate, and de-foaming rollers (193) remove bubbles from protective films.

In [37]:
len(result['novelty'])

676

In [32]:
# Phi-4 14b
display(Markdown(result['novelty']))

The invention is an auxiliary material automatic laminating machine, featuring a working table (11) with an electric slide rail (13) and locating fixture (14). A bracket (12) on the table connects to a screw rod and servo motor (15), facilitating movement. The top plate (16) links via a ball screw nut pair, while a cylinder (161) attaches to a bonding plate (17) equipped with vacuum suction discs (162). Positioned above the locating jig (14), the bonding plate slides on seats (19) supported by lifting springs and fixing seats (191) with de-foaming rollers (193). An electric push rod (192) connects to the sliding seat, enhancing bubble removal efficiency in laminating processes.

In [33]:
len(result['novelty'])

686

In [25]:
# deepseek-r1 14b
display(Markdown(result['novelty']))

An auxiliary material automatic laminating machine comprising a working table (11) with an electric slide rail (13) and locating jig (14). The top side of the working table is fixedly connected with a bracket (12), which is rotatably connected to a screw rod. The end of the screw rod is fixedly connected to a servo motor (15), which drives a top plate (16) via a ball screw nut pair. The top plate is connected to a cylinder (161), whose output end is fixedly connected to a bonding plate (17). The bonding plate has multiple vacuum suction discs (162) and is positioned above the locating jig. Sliding seats (19) with lifting springs are connected below the bonding plate, supporting a fixing seat (191) that rotates a de-foaming roller (193). An electric push rod (192) connects the bonding plate to the sliding seats.

In [26]:
len(result['novelty'])

822

In [7]:
# llama3.2:3b-instruct-fp16
display(Markdown(result['novelty']))

A working table; the middle part of the top side of the working table is fixedly connected with an electric slide rail; the electric slide rail is provided with a locating jig; the top side of the working table is fixedly connected with a bracket; the middle part of the bracket is rotatably connected with a screw rod; the end part of the screw rod is fixedly connected with a servo motor; said servo motor is fixedly connected with the bracket; the screw rod is connected with a top plate through a ball screw nut pair; the output end of the cylinder is fixedly connected with a bonding plate: multiple vacuum suckers are arranged at the bottom of the bonding plate; the bonding plate is located at the top part of the locating jig, the bubble of the protective film and the product is driven out from the centre to two sides by the movement of the bubble removing roller.

In [8]:
len(result['novelty'])

874